This notebook will compare model architecture, hyperparameters, inputs, and training/validation error

In [1]:
import xarray as xr
import numpy as np
from src.crossval import run_crossval

In [2]:
input_dir = "/glade/work/milesep/convective_outlook_ml"
target_dir = "data/processed_data"
stats_dir = "data/processed_data"

In [3]:
# compare all models, save training and validation curves along with corresponding model name, data level, model/training hyperparameters
model_names = ["cnn3d"]
levels = ["small"]
# Don't do linear regression with full dataset--too many parameters
for name, level in zip(model_names, levels):
    print(name, level)
    inputs = xr.open_zarr(f"{input_dir}/train_inputs_{level}.zarr")
    targets = xr.open_dataset(f"{target_dir}/train_targets.nc")
    stats = xr.open_dataset(f"{stats_dir}/daily_input_stats_{level}.nc")
    scores = run_crossval(inputs, targets, name, epochs = 40, level = level, restart = False)
    print(f"{name}: {np.mean(scores):.3f} ± {np.std(scores):.3f}")

cnn3d small

Fold 0:
Starting training from scratch
  Epoch 1/40 — Train Loss: 1.0185, Val Loss: 1.2766
  Epoch 2/40 — Train Loss: 0.9172, Val Loss: 1.2398
  Epoch 3/40 — Train Loss: 0.8749, Val Loss: 1.2244
  Epoch 4/40 — Train Loss: 0.9343, Val Loss: 1.2283
  Epoch 5/40 — Train Loss: 0.8505, Val Loss: 1.1941
  Epoch 6/40 — Train Loss: 0.8032, Val Loss: 1.2116
  Epoch 7/40 — Train Loss: 0.8234, Val Loss: 1.2445
  Epoch 8/40 — Train Loss: 0.8742, Val Loss: 1.2659
  Epoch 9/40 — Train Loss: 0.8565, Val Loss: 1.2728
  Epoch 10/40 — Train Loss: 0.7987, Val Loss: 1.2739
  Epoch 11/40 — Train Loss: 0.8273, Val Loss: 1.2765
  Epoch 12/40 — Train Loss: 0.7545, Val Loss: 1.3456
  Epoch 13/40 — Train Loss: 0.8140, Val Loss: 1.2380
  Epoch 14/40 — Train Loss: 0.9241, Val Loss: 1.3418
  Epoch 15/40 — Train Loss: 0.9107, Val Loss: 1.3346
  Epoch 16/40 — Train Loss: 0.7642, Val Loss: 1.2800
  Epoch 17/40 — Train Loss: 0.7809, Val Loss: 1.3856
  Epoch 18/40 — Train Loss: 0.7053, Val Loss: 1.3006
  E